**Prueba**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Cargar tus datos (asegúrate de tener un dataset adecuado)
df_test = pd.read_csv("/content/drive/MyDrive/GSCred/cs-test.csv", index_col=0)
df_train = pd.read_csv("/content/drive/MyDrive/GSCred/cs-training.csv", index_col=0)

y = df_train.SeriousDlqin2yrs
X = df_train.drop(['SeriousDlqin2yrs'], axis=1)


# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el clasificador XGBClassifier
xgb_classifier = XGBClassifier()

# Definir los hiperparámetros que quieres ajustar
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

# Realizar la búsqueda en cuadrícula para encontrar los mejores hiperparámetros
grid_search = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_

# Crear un nuevo modelo XGBClassifier con los mejores hiperparámetros
best_xgb_classifier = XGBClassifier(**best_params)

# Entrenar el modelo con los datos de entrenamiento
best_xgb_classifier.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = best_xgb_classifier.predict(X_test)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Imprimir resultados
print("Mejores hiperparámetros:", best_params)
print("Exactitud en el conjunto de prueba:", accuracy)
print("Informe de clasificación:")
print(classification_rep)



Mejores hiperparámetros: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Exactitud en el conjunto de prueba: 0.9386666666666666
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97     28044
           1       0.59      0.20      0.30      1956

    accuracy                           0.94     30000
   macro avg       0.77      0.60      0.63     30000
weighted avg       0.92      0.94      0.92     30000



In [2]:
x = df_test.drop(["SeriousDlqin2yrs"], axis=1)
y = best_xgb_classifier.predict_proba(x)[:, 1]
ids = x.index.values

#import numpy as np

#ids = np.arange(1,len(y)+1)
#submission = pd.DataFrame( {'Id': ids, 'Probability': y})
#submission.to_csv("/kaggle/working/submission.csv", index=False)